## Read the data

In [2]:
using DataFrames
using CSV
Cities = CSV.read("cities.csv")
head(Cities)

,CityId,X,Y
,Int64⍰,Float64⍰,Float64⍰
1,0,316.837,2202.34
2,1,4377.41,336.602
3,2,3454.16,2820.05
4,3,4688.1,2935.9
5,4,1010.7,3236.75
6,5,2474.23,1435.51


In [3]:
X = Array{Float64,1}(Cities[:X])   ## x-coordinates of the cities
Y = Array{Float64,1}(Cities[:Y])  ## y-coordinates of the cities
Data = [X';Y']
CityId = Array{Int,1}(Cities[:CityId]);

## Find out the prime cities

In [4]:
using Primes
PrimeCities = findall(isprime.(collect(1:size(Data,2)).-1));

## Basic Plan for TSP (Travelling Salesman Problem):
1. Use **KDE** to find the P points that represent the P densest places (P means peaks) <br>
2. Use **nearest neighbor principle** to partition all cities into clusters <br>
3. Use **ant colony optimization (ACO)** to figure out approximate shortest travelling path inside each cluster <br>
4. Combine the local paths inside each cluster to obtain the global path

### 1. Compute the KDE of cities

In [5]:
include("src\\f_KDE.jl")

Functions 'NormalDensity', 'f_KDE' imported


In [6]:
## Kernels = [NormalDensity(d, Data[i,:], H_MS) for i=1:n]
f_H = f_KDE(Data, 5000)

f_H (generic function with 1 method)

### 2. Find out the Modes of f_KDE

In [7]:
include("src\\Mode.jl")
Range = [minimum(Data[1,:]) maximum(Data[1,:]); minimum(Data[2,:]) maximum(Data[2,:])]
k = 10
N = 5000;

In [8]:
@time Peaks = Mode(f_H, Range, N = N, k = k);

104.554000 seconds (601.31 M allocations: 61.809 GiB, 8.41% gc time)


In [ ]:
using JLD

### 3. Compute the clusters

In [9]:
include("src\\VCluster.jl")

VCluster

In [10]:
Nc = length(Peaks) ## number of centers (peaks/clusters)
Cts = [[Peaks[i][1] for i=1:Nc]';[Peaks[i][2] for i=1:Nc]'] ## Cts = Centers
Cluster_Dic = VCluster(Cts, Data);

In [11]:
c_NP = 0 ## Cluster number of north pole
for c = 1:Nc
    if 1 in Cluster_Dic[c]
       c_NP = c
    end
end
c_NP

16

In [12]:
for k = 1:Nc
    println("CLuster $k size: $(length(Cluster_Dic[k]))")
end
sum([length(Cluster_Dic[k]) for k=1:Nc])

CLuster 1 size: 8933
CLuster 2 size: 16510
CLuster 3 size: 3605
CLuster 4 size: 4299
CLuster 5 size: 3423
CLuster 6 size: 6103
CLuster 7 size: 4315
CLuster 8 size: 2573
CLuster 9 size: 3546
CLuster 10 size: 10522
CLuster 11 size: 5982
CLuster 12 size: 3412
CLuster 13 size: 18071
CLuster 14 size: 8682
CLuster 15 size: 4456
CLuster 16 size: 9927
CLuster 17 size: 2678
CLuster 18 size: 7677
CLuster 19 size: 8345
CLuster 20 size: 6599
CLuster 21 size: 4553
CLuster 22 size: 6103
CLuster 23 size: 1769
CLuster 24 size: 3762
CLuster 25 size: 17591
CLuster 26 size: 8948
CLuster 27 size: 1090
CLuster 28 size: 6250
CLuster 29 size: 8045


197769

In [13]:
using Plots
plotly()
Nr = 5000 ## number of random cities
RandomCities = sample(collect(1:size(Data,2)), Nr, replace = false)
scatter([Data[1,1]], [Data[2,1]], label = "North Pole")
scatter()
@time for c = 1:Nc
    Cluster_k_Plot = intersect(RandomCities,Cluster_Dic[c])
    scatter!(X[Cluster_k_Plot],Y[Cluster_k_Plot], markersize = 1.5, label = "Cluster $c")
end
scatter!(Cts[1,:], Cts[2,:], label = "Peaks of KDE using k = $k")

  1.602826 seconds (2.84 M allocations: 145.476 MiB, 6.27% gc time)


### Cut the cluster containing NP into two subclusters

In [14]:
include("src\\CutHalf.jl")

CutHalf

In [15]:
CH = CutHalf(Data[:,Cluster_Dic[c_NP]], 1) ## cut the cluster containing NP into half
C_NP_Index = Cluster_Dic[c_NP] ## recall the indices in the cluster
Cluster_Dic[0] = sort(C_NP_Index[CH["With Center"]]) ## the subcluster containing NP
Cluster_Dic[c_NP] = sort(C_NP_Index[CH["Without Center"]]); ## the other subcluster

In [16]:
using Plots
plotly()
Nr = 5000 ## number of random cities
RandomCities = sample(collect(1:size(Data,2)), Nr, replace = false)
scatter([Data[1,1]], [Data[2,1]], label = "North Pole")
@time for c = 0:Nc
    Cluster_k_Plot = intersect(RandomCities, Cluster_Dic[c])
    scatter!(X[Cluster_k_Plot], Y[Cluster_k_Plot], markersize = 1.5, label = "Cluster $c")   
end
scatter!(Cts[1,:], Cts[2,:], label = "Peaks of KDE using k = $k")

  0.163637 seconds (101.20 k allocations: 13.183 MiB, 6.11% gc time)


### 4. Compute the inter-distances between clusters
Difficulty: computing using brute force is very computational costly

### An alternative solution in my mind is to
1. compute the convex hull of each cluster, and then
2. use the boundary information to reduce redundant computations. 

In [17]:
include("src\\Convex_Boundary_Inter_Distance.jl")

Convex_Boundary_Inter_Distance (generic function with 2 methods)

In [ ]:
Dist_C = zeros(Nc+1,Nc+1)
@time for k = 0:Nc-1
    for l = k+1:Nc
        C_k = Cluster_Dic[k] ## cluster k
        G_k = [X[C_k] Y[C_k]] ## coordinates of cluster k
        C_l = Cluster_Dic[l] ## cluster l
        G_l = [X[C_l] Y[C_l]] ## coordinates of cluster l
        Int_Dist = Convex_Boundary_Inter_Distance(G_k, G_l) ## Int_Dist = inter distance
        Dist_C[k+1,l+1] = Int_Dist["D_min"]
        Dist_C[l+1,k+1] = Int_Dist["D_min"]
    end
end
Dist_C

In [ ]:
Dist_C

In [48]:
## 
Cluster_Dic[k][Int_Dist["Group1"]]
Cluster_Dic[l][Int_Dist["Group2"]]
scatter(X[Cluster_Dic[k]], Y[Cluster_Dic[k]], markersize = 1, label = "Cluster $k")
scatter!(X[Cluster_Dic[l]], Y[Cluster_Dic[l]], markersize = 1, label = "Cluster $l")
scatter!([X[Cluster_Dic[k][Int_Dist["Group1"]]]], [Y[Cluster_Dic[k][Int_Dist["Group1"]]]], label = "CLosest Point $k")
scatter!([X[Cluster_Dic[l][Int_Dist["Group2"]]]], [Y[Cluster_Dic[l][Int_Dist["Group2"]]]], label = "CLosest Point $l")

In [51]:
Dist_C

38×38 Array{Float64,2}:
    0.0      2405.86     2235.37     …   909.047    1326.7      2172.98   
 2405.86        0.0        12.2039       778.055     118.0         6.32075
 2235.37       12.2039      0.0          283.299     291.567     212.466  
 1555.06      901.314     118.779          4.31932   705.948    1017.09   
 4590.27     1251.65     1679.64        2837.43     2281.19     1811.05   
 2844.13       99.3805      4.4391   …   920.572     870.637     762.418  
 3491.75     1503.74     1183.34        1647.15     2167.57     2099.8    
 3116.32      916.016     529.017       1158.47     1513.42     1444.83   
  604.539    1939.5      2093.67         915.232    1026.19     1292.6    
 2205.78     1500.96     1869.74        1655.86     1084.57      915.56   
  952.855    1399.05      799.304    …     3.99957   676.913    1435.59   
 1780.46      779.581    1011.37         907.485     204.72       39.3548 
  380.39     2578.93     2691.7         1399.47     1566.62     2015.71   
 

In [52]:
include("ACO_p_and_ACO_d.jl")

ACO_d (generic function with 6 methods)

In [71]:
Dist_C2 = Dist_C./maximum(Dist_C)

38×38 Array{Float64,2}:
 0.0          0.500138    0.464696     …  0.275799     0.451726  
 0.500138     0.0         0.002537        0.0245302    0.00131398
 0.464696     0.002537    0.0             0.0606119    0.0441681 
 0.323271     0.187368    0.0246922       0.146755     0.211436  
 0.954242     0.260197    0.349169        0.474221     0.376488  
 0.591248     0.0206596   0.000922817  …  0.180991     0.158494  
 0.725877     0.312603    0.245996        0.450602     0.436515  
 0.647832     0.190425    0.109974        0.314616     0.300356  
 0.125674     0.403191    0.43524         0.213328     0.26871   
 0.458546     0.312025    0.388687        0.225465     0.19033   
 0.198083     0.29084     0.166162     …  0.140719     0.298436  
 0.370128     0.162062    0.210246        0.0425578    0.00818123
 0.0790768    0.536118    0.55956         0.325675     0.419033  
 ⋮                                     ⋱                         
 0.546408     0.185374    0.286209        0.145702  

In [102]:
O = 3
G = 3
(L,P) = ACO_d(Dist_C2, O, G)

(0.4833430284656327, [3, 32, 37, 36, 22, 18, 9, 21, 13, 29  …  27, 24, 38, 20, 17, 12, 10, 33, 23, 3])

In [103]:
scatter(NewSample[Peaks,:][:,1], NewSample[Peaks,:][:,2], label = "Peaks")
scatter!()
plot!(NewSample[Peaks,:][P, 1], NewSample[Peaks,:][P, 2], label = "Shortest path, Length = $(round(L,digits = 2))")

## To deal with the "10-step prime city" constraint: